# **Extract High Quality Surface**
<hr>

In [1]:
from vmtk import vmtkscripts
from vmtk import pypes
import os
import numpy as np

In [2]:
# Change to directory where vmtk_functions and vmtk_filenames are located
os.chdir('/home/iagolessa/Documents/aneurysms/vmtkScripts')

# Import the necessary user's modules
import vmtk_functions
from vmtk_filenames import *

# Change it back
os.chdir('/home/iagolessa')

<hr>
**The next cell is used to define strings that holds path information about the loaded and saved data**

In [3]:
# Change the full path to the DICOM directory here
dicomDirectory = '/home/iagolessa/Documents/aneurysms/geometries/vrCases/originalData/45_IDO/IMAGE/'
dicomFirstFile = '1MF0EDOG' #'IM_00769.dcm' #None #'I1000000'

# Path to save the generated files when extracting the surface
# We reccomend to separate it from the DICOM original dir. 
# Note that inside casePath, this code will create different sub-directories to store different data types
casePath       = '/home/iagolessa/Documents/aneurysms/geometries/vrCases/extraCases/imageSubtracted/AquilionOne/vmtkReconstruction/759045/'

# Define subdirs
imagesDir       = casePath+'images/'
surfacesDir     = casePath+'surfaces/'
centerlinesDir  = casePath+'centerlines/'
meshesDir       = casePath+'meshes/'
parentVesselDir = casePath+"parentVessel/"

directories = [imagesDir,surfacesDir,centerlinesDir,meshesDir,parentVesselDir]

# Create the above directories if they do not exist
for directory in directories:
    if not os.path.isdir(directory):
        os.makedirs(directory)
        print(directory+' created.\n')

print('Files saved to:', casePath)
print('DICOM source directory: ',dicomDirectory)

Files saved to: /home/iagolessa/Documents/aneurysms/geometries/vrCases/extraCases/imageSubtracted/AquilionOne/vmtkReconstruction/759045/
DICOM source directory:  /home/iagolessa/Documents/aneurysms/geometries/vrCases/originalData/45_IDO/IMAGE/


<hr>
# **Extracting the Aneurysm Surface**

This section gather operations performed to *segment* a vessel and aneurysm surface (*vtkPolyData*) from a 3D image originated from a DSA or CTA scans (usually imported as a DICOM image) as a *vtkImageData*. The workflow is as follows: 

*Input: DICOM image (converted to vtk extension .vti)*
* Extraction of the VoI (*Volume of Interest*); 
* Segmentation with the Level Sets segmentation procedure and; 
* Surface extraction with the Marching Cubes algorithm.

*Output: triangulated surface file in vtk format .vtp or .stl*

--

Reading the DICOM directory to get image. 
To view image, use the function *viewImage(image)*

In [ ]:
imageDicom = vmtk_functions.readDicom(dicomDirectory,dicomFirstFile)

In [ ]:
vmtk_functions.viewImage(imageDicom)

In [ ]:
# Writing loaded DICOM to disk as .vti
# .vti is the native image format o VTK
vmtk_functions.writeImage(imageDicom,imagesDir+imageDicomFile)

In [ ]:
# The image morphology apply binary or grayscale morphology filter to
# the input image. For a better visualization with the MIP rendering
imageMorphology = vmtkscripts.vmtkImageMorphology()
imageMorphology.Image = imageDicom
imageMorphology.Operation = 'open'
imageMorphology.Execute()

# Debug
imageMorphology.PrintInputMembers()
imageMorphology.PrintOutputMembers()

vmtk_functions.viewImage(imageMorphology.Image)

In [ ]:
# Need to study a little bit this filter to know exactly 
# what kind of normalization it performs on the image
imageNormalizer = vmtkscripts.vmtkImageNormalize()
imageNormalizer.Image = imageDicom
imageNormalizer.Execute()

vmtk_functions.viewImage(imageNormalizer.Image)

In [ ]:
# imageReader = vmtkscripts.vmtkImageReader()

# imageReader.InputFileName = dicomDirectory+dicomFirstFile+'.dcm'
# imageReader.IORead()
# imageReader.PrintInputMembers()
# imageReader.PrintOutputMembers()
# imageReader.Execute()

# imageViewer = vmtkscripts.vmtkImageViewer()
# imageViewer.Image = imageReader.Image
# imageViewer.PrintInputMembers()
# imageViewer.PrintOutputMembers()
# imageViewer.Execute()

To find the aneurysm, it is easier to extract an 'initial surface' using the Marching Cubes algorithm instead of inspecting the 3D image. The function *initialSurface* encapsulates this task: it applies the Marching Cubes algorithm with a pre-defined level, and passes the largest connected surface. Note that the chosen level, depends on the image, and must be verified before by the user.

This procedure is kind of redundant for identifying the aneurysm compared with the MIP rendering, but is important to verify the correct level for the level sets segmentation algorithm.

In [ ]:
# Visualizing an initial surface
level = 2250
vmtk_functions.viewSurface(vmtk_functions.initialSurface(imageDicom,level))

In [ ]:
# Writing it to disk
vmtk_functions.writeSurface(
    vmtk_functions.initialSurface(imageDicom,level),
    surfacesDir+surfaceDicomFile,
    'binary'
)

In [ ]:
# BUG WITH VMTKRENDERER: WINDOW DOES NOT CLOSE
# ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
# surfaceOnImageViewer = vmtkscripts.vmtkRenderer()
# surfaceOnImageViewer.Execute()

# originalImage = vmtkscripts.vmtkImageViewer()
# originalImage.Image = imageDicom
# originalImage.vmtkRenderer = surfaceOnImageViewer.vmtkRenderer
# originalImage.Display = 0

# initialSurface = vmtkscripts.vmtkSurfaceViewer()
# initialSurface.Surface = vmtk_functions.initialSurface(imageDicom,level)
# initialSurface.vmtkRenderer = surfaceOnImageViewer.vmtkRenderer

# originalImage.Execute()
# initialSurface.Execute()

In [ ]:
# Check initial surface on original image
surfaceAgainstDicom = 'vmtkimagereader -ifile '+imagesDir+imageDicomFile+' ' \
         '--pipe vmtkrenderer --pipe vmtksurfaceviewer -ifile '+surfacesDir+surfaceDicomFile+' -display 0 -opacity 0.5 ' \
         '--pipe vmtkimageviewer -i @vmtkimagereader.o'
        
pypes.PypeRun(surfaceAgainstDicom)

If you already have a *.vti* extension image converted from a DICOM, skip directly to this cell

In [ ]:
imageDicom = readImage(imagesDir+imageDicomFile)
print('Reading vtkImageData object file.')
viewImage(imageDicom)

To work with a smaller portion of the vessels tree -- epsecially for the CFD simulation --, the next cell allows to cut a cubic portion of the image, known as the *volume of interest* (VOI).

In [ ]:
# Selection of VOI (centered in the aneurysm)
# The VOI is the region containing the aneurysm with its surrounding 
# vessels that will be extracted for a CFD simulation

# Object instantiation
imageVoiSelector = vmtkscripts.vmtkImageVOISelector()

# Object attributes
imageVoiSelector.Image = imageDicom
# Writing to disk parameters
imageVoiSelector.ImageOutputFileName = imagesDir+imageVoiFile

# Calling 'Execute' member function
imageVoiSelector.Execute()
imageVoiSelector.IOWrite()

# Debug
# imageVoiSelector.PrintInputMembers()
# imageVoiSelector.PrintOutputMembers()

vmtk_functions.viewSurface(vmtk_functions.initialSurface(imageVoiSelector.Image,level))

If already exists a VOI file, the next cell will load it, storing it in the imageVoiSelector object.

In [ ]:
imageVoiSelector = vmtkscripts.vmtkImageVOISelector()
imageVoiSelector.Image = readImage(imagesDir+imageVoiFile)

viewImage(imageVoiSelector.Image)

## Creating the sigmoid mask before going to Level Sets 

To create the proper mask on the image it is necessary to locate regions of bone and air. Air was included later because it also has a steep gradient, but the values in the air level set are opposite to those of bone. It has the same effect as bone on the segmentation of a vessel. To properly locate these regions, vmtklevelsetsegmentation is used.

In [ ]:
# Bone and air segmentation: give a try sometime

# extract level for bones
#vmtklevelsetsegmentation -ifile images/imageVoi.vti -ofeatureimagefile images/sigmoidMask/boneFeature300.vti -ofile images/sigmoidMask/boneLvlSet300.vti

# extract level for air
#vmtklevelsetsegmentation -ifile images/imageVoi.vti -ofeatureimagefile airFeature-50.vti  -ofile airLvlSet-50.vti

# Compose both images
#vmtkimagecompose -ifile bone_lvlset_500.vti -i2file air_lvlset_-50.vti -negatei2 1  -ofile bone_air_lvlset.vti

# Extract sigmoid feature image
#vmtkimagefeaturecorrection -ifile images/sigmoidMask/boneFeature300.vti -levelsetsfile images/sigmoidMask/boneAirLvlset.vti -scalefrominput 0 -ofile 
#images/sigmoidMask/sigmoidFeature.vti

# Testing 
<hr>
In this section, we apply filters to the input image, generating a feature image to be used with the level sets segmentation procedure.

***WARNING: The following operations consume a lot of RAM memory, use it knowing that they can completely freeze your computer***

We test the following filters:

    vmtkimagefeaturecorrection:	correct a feature image (e.g. remove influence of bone and/or air from CT-based feature images)
    vmtkimagefeatures: compute a feature image for use in segmentation
    vmtkimageobjectenhancement: compute a feature image for use in segmentation
    vmtkimageshiftscale: shift and scale the intensity of an image and cast it to a specified type
    vmtkimagevesselenhancement: compute a feature image for use in segmentation

In [ ]:
# Instantiate imagecast
imageCast = vmtkscripts.vmtkImageCast()

imageCast.Image = imageVoiSelector.Image
imageCast.OutputType = 'float'
imageCast.Execute()

# Debug
# imageCast.PrintInputMembers()
# imageCast.PrintOutputMembers()

# ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
vesselEnhancer = vmtkscripts.vmtkImageVesselEnhancement()

vesselEnhancer.Image = imageCast.Image

# Options = ['frangi', 'sato', 'ved', 'vedm']
vesselEnhancer.Method = 'sato'

# Sigma setup
vesselEnhancer.SigmaMin = 1.0
vesselEnhancer.SigmaMax = 1.0
vesselEnhancer.NumberOfSigmaSteps = 1
# Options = ['equispaced','logarithmic']
vesselEnhancer.SigmaStepMethod = 'equispaced'

vesselEnhancer.ScaledVesselness = 1
vesselEnhancer.Alpha1 = 0.5
vesselEnhancer.Alpha2 = 2.0
vesselEnhancer.Alpha = 1.0
vesselEnhancer.Beta = 1.0
vesselEnhancer.Gamma = 0.5
vesselEnhancer.C = 1e-06
vesselEnhancer.TimeStep = 0.01
vesselEnhancer.Epsilon = 0.01

vesselEnhancer.Execute()

# Debug
vesselEnhancer.PrintInputMembers()
vesselEnhancer.PrintOutputMembers()

In [ ]:
vmtk_functions.viewImageMIP(vesselEnhancer.Image)

<hr>

## **Level sets segmentation procedure**

Level sets segmentation procedure: usually the fastest (and that sometimes leads to good quality surfaces) is the *isosurface* method, it requires only the level, outputing a complete surface correspondig to that level. The value 2500 used below is a value that usually leads directly to the surface close to the desired one. However, this situation can be the case only for these images; there may obviously be images with different levels (good question for automation: how to select the correct level?)

The first cell below generates an initial image -- *initializationImage* -- to be used with the Level Sets segmentation procedure, as it is required by the method. This initial image is generated using the *isosurface* method.

In [ ]:
# Object instantiantion
initializationImage = vmtkscripts.vmtkImageInitialization()

# Select existing image
try:
    initializationImage.Image = imageVoiSelector.Image
except:
    initializationImage.Image = imageDicom
    
initializationImage.Method = 'isosurface'
initializationImage.IsoSurfaceValue = level
initializationImage.Interactive = 0
initializationImage.InitialLevelSetsOutputFileName = imagesDir+imageInitialLevelSetsFile

initializationImage.Execute()
# initializationImage.IOWrite()
# viewImage(initializationImage.InitialLevelSets)

# Debug
# initializationImage.PrintInputMembers()
# initializationImage.PrintOutputMembers()

In [ ]:
# ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~ #
# If image was generated from dicom above
imageLevelSets = vmtkscripts.vmtkLevelSetSegmentation()

# Input image
imageLevelSets.Image = initializationImage.Image
imageLevelSets.InitialLevelSets = initializationImage.InitialLevelSets

# Vessel enhanced image used as a feature image
# imageLevelSets.FeatureImage = vesselEnhancer.Image


imageLevelSets.LevelSetsOutputFileName = imagesDir+imageLevelSetsFile
# Once the branch segmented, the parameters that have led to good agreement between the extracted level and image are:
# -- NumberOfIterations = 300
# -- PropagationScaling = 0.5: usually 0.0, but the isosurface method does not get the gradients very well
# -- CurvatureScaling = 0.1: usually 0.0, it is important not to set very high because it can degenerate the surface
#                            however, i noted that setting slighty above 0.0 remove high curvature parts on the surface
# -- AdvectionScaling = 1.0: usually 1.0 is perfect

imageLevelSets.NumberOfIterations = 300
imageLevelSets.PropagationScaling = 0.5
imageLevelSets.CurvatureScaling = 0.1
imageLevelSets.AdvectionScaling = 1.0
imageLevelSets.SmoothingIterations = 20

# Execute Level Sets
imageLevelSets.Execute()
# imageLevelSets.IOWrite()

# Debug
# imageLevelSets.PrintInputMembers()
imageLevelSets.PrintOutputMembers()

In [ ]:
vmtk_functions.viewImage(imageLevelSets.LevelSets)

In [ ]:
# ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~ #
# Converting VOI image to surface using Marching Cubes
surfaceMarchingCubes = vmtkscripts.vmtkMarchingCubes()

# The input image to Marching Cubes must be the Level Sets image
surfaceMarchingCubes.Image = imageLevelSets.LevelSets

# surfaceMarchingCubes.ImageInputFileName = imagesDir+'imageLevelSets.vti'
# surfaceMarchingCubes.IORead()
# Writing parameters
# surfaceMarchingCubes.SurfaceOutputFileName = surfacesDir+surfaceVoiFile

# Parameters
surfaceMarchingCubes.Level = 0.1# increasing the value to inflate the surface
# surfaceMarchingCubes.PrintInputMembers()
surfaceMarchingCubes.Execute()

# Visualizing output surface
# surfaceMarchingCubes.IOWrite()
# surfaceMarchingCubes.OutputText('Initial surface wrote to '+surfaceMarchingCubes.SurfaceOutputFileName)

# Run vmtksurfacetriangle to prevent pathed surfaces from Marching Cubes
# before using the vmtksurfaceconnected
surfaceTriangle = vmtkscripts.vmtkSurfaceTriangle()
surfaceTriangle.Surface = surfaceMarchingCubes.Surface
surfaceTriangle.Execute()

# Extract largest connected surface
surfaceConnected = vmtkscripts.vmtkSurfaceConnectivity()
surfaceConnected.Surface = surfaceTriangle.Surface
surfaceConnected.SurfaceOutputFileName = surfacesDir+surfaceVoiFile
surfaceConnected.Execute()
surfaceConnected.IOWrite()

In [ ]:
vmtk_functions.viewSurface(surfaceConnected.Surface)

In [ ]:
# potentialFit = vmtkscripts.vmtkPotentialFit()

# potentialFit.PrintInputMembers()
# potentialFit.Surface = surfaceConnected.Surface
# potentialFit.Image = imageLevelSets.LevelSets

# potentialFit.Execute()
# potentialFit.PrintOutputMembers()
# viewSurface(potentialFit.Surface)

Final comparison between the extracted surface and the initial image. It uses the *pypes* vmtk module to compare both image and surface in a single renderer for inspection.

In [ ]:
# TRY WITH VMTKRENDERER
myPipe = 'vmtkimagereader -ifile '+imagesDir+imageDicomFile+' ' \
         '--pipe vmtkrenderer --pipe vmtksurfaceviewer -ifile '+surfacesDir+surfaceVoiFile+' -display 0 -opacity 0.5 ' \
         '--pipe vmtkimageviewer -i @vmtkimagereader.o'
        
print(myPipe)
pypes.PypeRun(myPipe)

In [ ]:
# RENDERER WITH PROBLEMS

#renderer = vmtkscripts.vmtkRenderer()
# renderer.Execute()

# surfaceExtracted = vmtkscripts.vmtkSurfaceViewer()
# surfaceExtracted.Surface = surfaceConnected.Surface
# surfaceExtracted.vmtkRenderer = renderer
# surfaceExtracted.Execute()

# imageExtracted = vmtkscripts.vmtkImageViewer()
# imageExtracted.Image = imageDicom
# imageExtracted.vmtkRenderer = renderer
# imageExtracted.Execute()

<hr>
# **Aneurysm Surface Quality Treatment**

In general, the segmented (triangulated) surface does not have a very good quality (degenerated and/or warped triangles, for example). 
Hence, some operations must be performed to treat the surface before using it to generate a good **volume mesh** for a CFD simulation.
VMTK provides such operations, as: 

* Smoothing the surface, i.e., creating good quality triangles on it;
* Subdividing (surface refinement): increase the detail level of the surface;
* Clipping the surface to remove surrounding vessels;
* Capping i.e. closing the surface with caps;
* Surface remeshing: completely change the surface with high quality triangles;

among other operations. The following cells gather all these possible operations.

<hr>
**Input surface: the final surface of the Marching Cubes procedure or the chosen surface for treatment**

In [ ]:
# If the segmentation procedure has just been finished
# and the "surfaceVoI" is available in memory

surfaceInput = vmtk_functions.readSurface(surfacesDir+surfaceVoiFile)
vmtk_functions.viewSurface(surfaceInput)

# Smoothing the segmented surface
surfaceSmoothed = vmtk_functions.smoothSurface(surfaceConnected.Surface)

# Subdividing surface triangles
surfaceSubdivided = vmtk_functions.subdivideSurface(surfaceSmoothed)
vmtk_functions.viewSurface(surfaceSubdivided)

In [ ]:
vmtk_functions.writeSurface(surfaceSubdivided,surfacesDir+surfaceSmoothedFile,'ascii')

# **TESTING**
<hr>

In [4]:
surfaceInput = vmtk_functions.readSurface(surfacesDir+'surfaceWithClipArray.vtp')
# vmtk_functions.viewSurface(surfaceInput)

Reading VTK XML surface file.


In this cell we try to use the loop extractor script. I still do not get what is the main purpose of this script, seems not very useful at least for our purposes.

In [ ]:
loopExtractor = vmtkscripts.vmtkSurfaceLoopExtraction()

# loopExtractor.Surface = surfaceInput
loopExtractor.SurfaceInputFileName = surfacesDir+'surfaceWithClipArray.vtp'
loopExtractor.IORead()
loopExtractor.LoopOutputFileName = surfacesDir+'loop.vtp'

loopExtractor.Execute()
loopExtractor.IOWrite()
loopExtractor.PrintInputMembers()
loopExtractor.PrintOutputMembers()

In [ ]:
vmtk_functions.viewSurface(loopExtractor.Loop)

In [ ]:
loopClipper = vmtkscripts.vmtkSurfaceClipLoop()

loopClipper.Surface = loopExtractor.Surface
loopClipper.Loop = loopExtractor.Loop
loopClipper.Execute()

loopClipper.PrintInputMembers()
loopClipper.PrintOutputMembers()

In [ ]:
vmtk_functions.viewSurface(loopClipper.Surface)

<hr> 
In the following cells, we try to automatically improve the surface obtained above and to generate a surface in a *region of interest* (ROI) with open inlets and outlets where boundary conditions will be applied in a CFD simulation. Some comments on the process are given here.

The first task would be to reduce the above surface to the ROI volume, which can be performed in the next cell. The widget type can be changed between box or sphere, however I think that the best approach is to use box because it has more degrees of freedom than a sphere, hence resulting in more flexibility to extract the ROI. In the sequence, we apply the *vmtksurfaceconnectivity* to avoid separate portions of the clipped surface to be passed along the process. 

In [ ]:
# Clipping the surface
roiSelector = vmtkscripts.vmtkSurfaceClipper()

roiSelector.Surface = surfaceInput
# roiSelector.SurfaceInputFileName = surfacesDir+surfaceRemeshedSmFile
# roiSelector.IORead()
roiSelector.InsideOut = 1
roiSelector.WidgetType = 'box'
roiSelector.Execute()

# Extract largest connected surface
surfaceConnected = vmtkscripts.vmtkSurfaceConnectivity()
surfaceConnected.Surface = roiSelector.Surface
surfaceConnected.Execute()

In [ ]:
vmtk_functions.writeSurface(surfaceConnected.Surface,surfacesDir+'test.vtp','binary')

Then, the ROI surface is passed to the region drawing script where the operator must define a loop around the vessels and the regions that must be cut, either because it does not correspond to reality (spourious lateral connections between vessels originating from the segmentation procedure) or to extract vessels that will not be simulated.

**The script only works with closed contours!**

The resulting surface with the cutting array is then passed to clipper to clip on middle values of the resulting array. But before the array is smoothed to result in a smoother cut from the clipping procedure, which is better when the surface is fixed. Sometimes, for arrays in loop regions, it is better to skip the smoothing operations because it can sooth out the middle value, which would cause the clipping on this region to not occur. 

After the clipping operation, it is wise to remesh the surface to get better quality contours on the hole of the surface because the surfaceFixer works internally with the *vmtksurfacecapper* script using the smooth method. 

In [ ]:
cutRegionsMarker = vmtkscripts.vmtkSurfaceRegionDrawing()

cutRegionsMarker.Surface = surfaceInput #surfaceConnected.Surface
cutRegionsMarker.Execute()

# Debug
# cutRegionsMarker.PrintInputMembers()
# cutRegionsMarker.PrintOutputMembers()

In [ ]:
cutRegionsMarker.SurfaceOutputFileName = surfacesDir+'surfaceWithClipArray.vtp'
cutRegionsMarker.IOWrite()

In [ ]:
# Since the array created by RegionDrawing is discontinous
# we use the array smoothing
cutRegionsSmoother = vmtkscripts.vmtkSurfaceArraySmoothing()
cutRegionsSmoother.Surface = cutRegionsMarker.Surface
cutRegionsSmoother.SurfaceArrayName = cutRegionsMarker.ContourScalarsArrayName
# General options
cutRegionsSmoother.Connexity = 1
cutRegionsSmoother.Relaxation = 1.0
cutRegionsSmoother.Iterations = 1
cutRegionsSmoother.Execute()

In [ ]:
cutRegionsSmoother.SurfaceOutputFileName = surfacesDir+'surfaceWithSmoothClipArray.vtp'
cutRegionsSmoother.IOWrite()
# Debug
# cutRegionsSmoother.PrintInputMembers()
# cutRegionsSmoother.PrintOutputMembers()

In [ ]:
# View surface
surfaceViewer = vmtkscripts.vmtkSurfaceViewer()
surfaceViewer.Surface = cutRegionsSmoother.Surface
surfaceViewer.ArrayName = cutRegionsSmoother.SurfaceArrayName
surfaceViewer.Legend = 1
surfaceViewer.Execute()

In [5]:
# CLIPPER IN VERSION 1.4 WITH POSSIBLE BUG IN INTERACTIVE PYTHON
# Initial Clipper to revome excess of surface
surfaceArrayClipper = vmtkscripts.vmtkSurfaceClipper()

surfaceArrayClipper.Surface = surfaceInput #cutRegionsMarker.Surface #cutRegionsSmoother.Surface
# surfaceArrayClipper.SurfaceInputFileName = surfacesDir+'surfaceWithClipArray.vtp'
# surfaceArrayClipper.IORead()
# surfaceClipper.InsideOut = 0
# surfaceClipper.WidgetType = 'box'
surfaceArrayClipper.Interactive = 0
surfaceArrayClipper.ClipArrayName = 'ContourScalars' #cutRegionsMarker.ContourScalarsArrayName #cutRegionsSmoother.SurfaceArrayName
surfaceArrayClipper.ClipValue = 0.5#*(cutRegionsMarker.InsideValue + \
#                                      cutRegionsMarker.OutsideValue)
surfaceArrayClipper.Execute()

# Debug
# surfaceArrayClipper.PrintInputMembers()
# surfaceArrayClipper.PrintOutputMembers()

# Extract largest connected surface
surfaceConnected = vmtkscripts.vmtkSurfaceConnectivity()
surfaceConnected.Surface = surfaceArrayClipper.Surface
surfaceConnected.Execute()

In [ ]:
# Check if is correct (debug)
vmtk_functions.viewSurface(surfaceConnected.Surface)

In [ ]:
# surfaceArrayClipper.SurfaceOutputFileName = surfacesDir+'surfaceArrayClipped.vtp'
# surfaceArrayClipper.IOWrite()

In [6]:
# Simple remesh procedure to increase quality at cut lines
surfaceRemesh = vmtkscripts.vmtkSurfaceRemeshing()

surfaceRemesh.Surface = surfaceConnected.Surface
surfaceRemesh.ElementSizeMode = "edgelength"
surfaceRemesh.TargetEdgeLength = 0.2
surfaceRemesh.SurfaceOutputFileName = surfacesDir+surfaceRemeshedFile
surfaceRemesh.Execute()
surfaceRemesh.IOWrite()
# surfaceRemesh.Execute()

Writing VTK XML surface file.


In [ ]:
vmtk_functions.viewSurface(surfaceRemesh.Surface)

With the *surfaceFixer* object I try finally to close the above surface, clipped with an array value, however for curved cuts it is imporssible to correctly close the surface following the vessels walls.

In [ ]:
surfaceFixer = vmtkscripts.vmtkSurfaceCapper()

surfaceFixer.Surface = surfaceRemesh.Surface
# Capping Method = 'simple', 'centerpoint', 'smooth', 'annular', 'concaveannular'
surfaceFixer.Method = 'smooth'
surfaceFixer.ConstraintFactor = 0.2
surfaceFixer.NumberOfRings = 6
surfaceFixer.Interactive = 0

surfaceFixer.Execute()

# Debug
# surfaceFixer.PrintInputMembers()
# surfaceFixer.PrintOutputMembers()

In [ ]:
surfaceSmoothed = vmtk_functions.smoothSurface(surfaceFixer.Surface)
vmtk_functions.viewSurface(surfaceSmoothed)
vmtk_functions.writeSurface(surfaceSmoothed,surfacesDir+'surfaceFixed.stl','ascii')

<hr>
**The remeshing procedure is very important to get a surface with *high* quality triangles. I also noted that this script is very robust even with large and complex vessels surface network.**

In [ ]:
# Remeshing the surface with quality triangles

surfaceRemesh = vmtkscripts.vmtkSurfaceRemeshing()

surfaceRemesh.Surface = surfaceSmoothed
# surfaceRemesh.SurfaceInputFileName = surfaceSmoothed
# surfaceRemesh.IORead()

surfaceRemesh.ElementSizeMode = "edgelength"
surfaceRemesh.TargetEdgeLength = 0.1

# If ElementSizeMode = "edgelenghtarray"
# Then use DistanceToCenterlines array for radius adaptative surface mesh
# surfaceRemesh.ElementSizeMode = "edgelengtharray"
# surfaceRemesh.TargetEdgeLengthArrayName = "DistanceToCenterlines"
# surfaceRemesh.TargetEdgeLengthFactor = 0.2

# If ElementSizeMode = "area"
# surfaceRemesh.TargetArea = 0.1

# Setup
#surfaceRemesh.PrintInputMembers()
# surfaceRemesh.PrintOutputMembers()

# Write parameters
surfaceRemesh.SurfaceOutputFileName = surfacesDir+surfaceRemeshedFile

surfaceRemesh.Execute()
# surfaceRemesh.IOWrite()

In [ ]:
# Smoothing procedure after remeshing
# (even after the remeshing it is a good idea to smooth the surface to decrease the feature angles)
surfaceRmSmoothed = vmtk_functions.smoothSurface(surfaceRemesh.Surface)
vmtk_functions.viewSurface(surfaceRmSmoothed)

In [ ]:
vmtk_functions.writeSurface(surfaceRmSmoothed,surfacesDir+'surfaceFixedRm.stl','ascii')

In [ ]:
#MAYBE IS BETTER TO PUT THE CENTERLINE CONSTRUICTION ABOVE THE REMESHING PROCEDURE TO CHECK NON MANIFOLDNESS
# computing centerlines to use with vmtksurfaceendclipper
centerlines = vmtkscripts.vmtkCenterlines()

centerlines.Surface = surfaceRmSmoothed
centerlines.AppendEndPoints = 1
centerlines.Execute()

# Debug
# centerlines.PrintInputMembers()
# centerlines.PrintOutputMembers()


# Computing centerlines Frenet system
centerlineGeometry = vmtkscripts.vmtkCenterlineGeometry()

centerlineGeometry.Centerlines = centerlines.Centerlines
# centerlineGeometry.CenterlinesOutputFileName = centerlinesDir+'centerlineGeometry.vtp' # centerlineGeometryFile
centerlineGeometry.LineSmoothing = 0
centerlineGeometry.Execute()
# centerlineGeometry.IOWrite()

# Debug
# centerlineGeometry.PrintInputMembers()
# centerlineGeometry.PrintOutputMembers()



# And then execute end clipper
# SOMETINES DOES NOT WORK
surfaceEndClipper = vmtkscripts.vmtkSurfaceEndClipper()

surfaceEndClipper.Surface = centerlines.Surface
# surfaceEndClipper.SurfaceInputFileName = surfacesDir+'surfaceRemeshedSmBlended.stl'
# surfaceEndClipper.IORead()

# Setup
surfaceEndClipper.CenterlineNormals = 1
surfaceEndClipper.Centerlines = centerlines.Centerlines
surfaceEndClipper.FrenetTangentArrayName = centerlineGeometry.FrenetTangentArrayName

# surfaceEndClipper.SurfaceOutputFileName = surfacesDir+surfaceEndClippedFile
surfaceEndClipper.Execute()

vmtk_functions.viewSurface(surfaceEndClipper.Surface)
# surfaceEndClipper.IOWrite()

# Debug
surfaceEndClipper.PrintInputMembers()
surfaceEndClipper.PrintOutputMembers()

In [ ]:
# Final comparison between the extracted remeshed surface and the initial VOI image

myPipe = 'vmtkimagereader -ifile '+imagesDir+imageDicomFile+' ' \
         '--pipe vmtkrenderer --pipe vmtksurfaceviewer -ifile '+surfacesDir+surfaceRemeshedFile+' -display 0 -opacity 0.3 ' \
         '--pipe vmtkimageviewer -i @vmtkimagereader.o'
        
print(myPipe)
pypes.PypeRun(myPipe)

In [ ]:
# Clipping the surface
surfaceClipper = vmtkscripts.vmtkSurfaceClipper()
# surfaceClipper.Surface = 
surfaceClipper.SurfaceInputFileName = surfacesDir+surfaceRemeshedSmFile
surfaceClipper.IORead()
surfaceClipper.InsideOut = 0
surfaceClipper.WidgetType = 'box'
surfaceClipper.Execute()

surfaceClipper.SurfaceOutputFileName = surfacesDir+surfaceClippedFile
surfaceClipper.IOWrite()

# Writer surface in other format
# writeSurface(surfaceClipper.Surface,casePath+"surfaces/surfaceClipped.stl","ascii")

In [ ]:
# FLOW EXTENSIONS NOT WORKING HERE: POSSIBLE BUG THAT GENERATES ZOMBIE WINDOWS OF THE PROCEDURE
# SO USE IT WITH Interaction = 0

# # Compute centerlines for flow extensions procedure
# Only needed if the 'centerlinedirection' is used
# centerlines = vmtkscripts.vmtkCenterlines()

# # centerlines.Surface = surfaceRmSmoothed
# centerlines.SurfaceInputFileName = surfacesDir+surfaceFinalFile
# centerlines.IORead()
# # centerlines.AppendEndPoints = 1
# centerlines.Execute()

# ~~~~~~~~~~~~~~~~~~~~~~
# Adding flow extensions
surfaceFlowExtensions = vmtkscripts.vmtkFlowExtensions()

# surfaceFlowExtensions.Surface = centerlines.Surface
# surfaceFlowExtensions.Centerlines = centerlines.Centerlines

surfaceFlowExtensions.SurfaceInputFileName = surfacesDir+surfaceClippedFile
# surfaceFlowExtensions.CenterlinesInputFileName = centerlinesDir+'centerlines.vtp'
surfaceFlowExtensions.IORead()
surfaceFlowExtensions.SurfaceOutputFileName = surfacesDir+surfaceWithFlowExtFile

# Setup
surfaceFlowExtensions.InterpolationMode = 'thinplatespline' # or linear
surfaceFlowExtensions.ExtensionMode = 'boundarynormal' # or centerlinedirection
# boolean flag which enables computing the length of each 
# flowextension proportional to the mean profile radius
surfaceFlowExtensions.AdaptiveExtensionLength = 1 # (bool)

# The proportionality factor is set through 'extensionratio'
surfaceFlowExtensions.ExtensionRatio = 1

# Controls how far into the centerline the algorithm looks 
# for computing the orientation of the flow extension.
surfaceFlowExtensions.CenterlineNormalEstimationDistanceRatio = 1


surfaceFlowExtensions.Interactive = 0
surfaceFlowExtensions.TransitionRatio = 0.5
surfaceFlowExtensions.AdaptiveExtensionRadius = 1
surfaceFlowExtensions.AdaptiveNumberOfBoundaryPoints = 1
surfaceFlowExtensions.TargetNumberOfBoundaryPoints = 50
surfaceFlowExtensions.Sigma = 1.0


# surfaceFlowExtensions.PrintInputMembers()
# surfaceFlowExtensions.PrintOutputMembers()
surfaceFlowExtensions.Execute()
surfaceFlowExtensions.IOWrite()

In [ ]:
vmtk_functions.viewSurface(surfaceFlowExtensions.Surface)

In [ ]:
# Remeshing procedure
flowExtRemesh = vmtkscripts.vmtkSurfaceRemeshing()

flowExtRemesh.Surface = surfaceFlowExtensions.Surface
flowExtRemesh.ElementSizeMode = "edgelength"
flowExtRemesh.TargetEdgeLength = 0.1
# flowExtRemesh.SurfaceOutputFileName = surfacesDir+surfaceRemeshedFile
flowExtRemesh.Execute()
vmtk_functions.viewSurface(flowExtRemesh.Surface)
# flowExtRemesh.IOWrite()

# Smoothing procedure after remeshing
# (even after the remeshing it is a good idea to smooth the surface to decrease the feature angles)
surfaceFlowSmoothed = vmtk_functions.smoothSurface(flowExtRemesh.Surface)
vmtk_functions.viewSurface(surfaceFlowSmoothed)
vmtk_functions.writeSurface(surfaceFlowSmoothed,surfacesDir+surfaceFinalFile,'binary')

In [ ]:
# Pype to add flow extension
flowExtAddPipe = 'vmtksurfacereader -ifile '+surfacesDir+surfaceRemeshedSmFile+' ' \
                 '--pipe vmtkcenterlines -seedselector openprofiles ' \
                 '--pipe vmtkflowextensions -adaptivelength 1 -extensionratio 3 -normalestimationratio 1 -interactive 0 '\
                 '--pipe vmtksurfacewriter -ofile '+surfacesDir+surfaceWithFlowExt

print(myPipe)
pypes.PypeRun(myPipe)

Finally, capping the high quality surface to proceed with the orientation and splitting the surface before send to SnappyHexMesh procedure.
The capping procedure close the 'wholes' on the surface, corresponding to inlets and outlets of the model geometry (used for boundary conditions for CFD simulations).
It is important to note that the capping procedure also identify each cap and the vascular surface with ids, to allow splitting of each patch.

In [ ]:
# Capping the surface
surfaceCapper = vmtkscripts.vmtkSurfaceCapper()

surfaceCapper.Surface = surfaceClipper.Surface
# surfaceCapper.SurfaceInputFileName = surfacesDir+surfaceClippedFile
# surfaceCapper.IORead()
surfaceCapper.SurfaceOutputFileName = surfacesDir+surfaceCappedFile

# Capping Method = 'simple', 'centerpoint', 'smooth', 'annular', 'concaveannular'
surfaceCapper.Method = 'centerpoint'
surfaceCapper.NumberOfRings = 8
# surfaceCapper.ConstraintFactor = 0.0
surfaceCapper.Interactive = 0

surfaceCapper.Execute()
surfaceCapper.IOWrite()

# Debug
# surfaceCapper.PrintInputMembers()
# surfaceCapper.PrintOutputMembers()

The final surface obtained here is a capped surface of the aneurysm and surrounnding vessels.
The surface can have its quality increased by using other tools such as Blender and Paraview, before send it to meshing using SnappyHexMesh.

<hr>
# **Surface Properties**

Below, we compute some properties of the final surface:
* Surface Curvature; 
* Surface Normals;
* Volume and surface area;
* Boundary Inspector;

In [ ]:
# Surface curvature array
surfaceCurvature = vmtkscripts.vmtkSurfaceCurvature()

surfaceCurvature.Surface = surfaceCapper.Surface
surfaceCurvature.CurvatureType = 'gaussian'
surfaceCurvature.AbsoluteCurvature = 1
surfaceCurvature.MedianFiltering = 1
surfaceCurvature.BoundedReciprocal = 1
surfaceCurvature.Execute()

In [ ]:
surfaceNormals = vmtkscripts.vmtkSurfaceNormals()

surfaceNormals.Surface = surfaceCurvature.Surface
surfaceNormals.SurfaceOutputFileName = surfacesDir+"surfaceNormals.vtp"
surfaceNormals.Execute()
surfaceNormals.IOWrite()

In [ ]:
# Calculate volume and area of surface
surfaceMassProperties = vmtkscripts.vmtkSurfaceMassProperties()

# surfaceMassProperties.Surface = aneurysm # aneurysmClipper.Surface
surfaceMassProperties.SurfaceInputFileName = surfacesDir+surfaceCappedFile
surfaceMassProperties.IORead()
surfaceMassProperties.Execute()

viewSurface(surfaceMassProperties.Surface)
print('Surface area: '+str(surfaceMassProperties.SurfaceArea)+' mm2')
print('Volume: '+str(surfaceMassProperties.Volume)+' mm3')

In [ ]:
# SCRIPTS TO INVESTIGATE
# vmtklocalgeometry
# vmtkendpointextractor
# vmtkentityrenumber
# vmtksurfacecenterlineprojection

In [ ]:
# ESTE SCRIPT NAO PARECE SFUNCIONAR MUITO BEM, ANTES OU DEPOIS DE SMOOTHING
# TESTAR EM OUTROS CASOS (TESTE FEITO NA VERSAO EM DESENVOLVIMETO 1.3, TESTAR NA 1.2)

# surfaceKiteRemoval = vmtkscripts.vmtkSurfaceKiteRemoval()

# surfaceKiteRemoval.Surface = inputSurface
# surfaceKiteRemoval.SizeFactor = 0.1
# surfaceKiteRemoval.PrintInputMembers()
# surfaceKiteRemoval.PrintOutputMembers()
# surfaceKiteRemoval.Execute()

# viewSurface(surfaceKiteRemoval.Surface)